# HoloViews Basics
Elements and Containers
- Some examples are taken from HoloViews [tutorials](#)


## Load Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
    
from pathlib import Path
import pdb

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
from holoviews import opts

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn

In [ ]:
# Visualization default options
H,W, = 500,500
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Image('mask', alpha=0.3),

    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Path(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Tiles(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
)

## 1. Elements


### General format

hv.`<ElementType>` ( `<data object>`, `<dimension annotations>` )


> **Visualization of a dependent variable against an independent variable**  
> - key dimension (typically the *x* axis in a plot), and usually have one or more value dimension(s) (often the *y* axis) that may or may not be indexable depending on the implementation.    
> - The key dimensions are normally the parameter settings for which things are measured, and the value dimensions are the data points recorded at those settings.   
> - As described in the [Columnar Data tutorial](Columnar_Data.ipynb), the data can be stored in several different internal formats, such as a NumPy array of shape (N, D), where N is the number of samples and D the number of dimensions. A somewhat larger list of formats can be accepted, including any of the supported internal formats, or  
    1. As a list of length N containing tuples of length D.
    2. As a tuple of length D containing iterables of length N.   
>From. HoloViews documentation 

### `hv.Points`
$x$ and $y$ are **independent** varialbe, known as "key dimensions" (`kdims`) in HoloViews

![hv-points](../assets/hv/hv-points.png)

In [ ]:
points = [(0.1*i, np.sin(0.1*i)) for i in range(100)]
hv.Points(points)

In [ ]:
hv.Points(points).dimensions()

### `hv.Curve`
`hv.Curve` represents a set of sample pairs $(x,y)$ that are assumed to have a deterministic relation. The plotted values will be connected because `hv.Curve` represents the relationship (ie. a function) between x and y.

In [ ]:
points = [(0.1*i, np.sin(0.1*i)) for i in range(100)]
hv.Curve(points)

In [ ]:
# Exercise 
# Make a Curve representing a function f s.t.
# y = f(x) = sigmoid(x) 
# Note: sigmoid(x) = 1 / (1+e^(-x))
# You can set the range of x (ie. domain) as you'd like

In [ ]:
# Soln
def sigmoid(x):
    return 1/(1+np.exp(-x))
xs = np.linspace(-10,10,100)
ys = sigmoid(xs)
sigmoid_curve = hv.Curve( (xs, ys) )
sigmoid_curve

### `hv.Scatter`
`Scatter` element expresses a dependent relationship between two variables.
$x$ is an **independent** variable, and $y$ is an **dependent** variable, ie. a function of $x$.

In [ ]:
hv.Scatter(points)

`hv.Scatter` vs. `hv.Points`

They are semantically different! 

In [ ]:
for o in [hv.Points(points,name="Points "), hv.Scatter(points,name="Scatter")]:
    for d in ['key','value']:
        print("%s %s_dimensions: %s " % (o.name, d, o.dimensions(d,label=True)))

### `hv.HLine`, `hv.VLine`
Holoviews components can be overlaid onto other components using `*` operator.
`hv.HLine` and `hv.VLine` are useful elements for adding annotations to other elements like `Curve`and `Image`

In [ ]:
hline_0 = hv.HLine(y=0.).opts(color='red', line_dash='dashed', line_width=1)
hline_1 = hv.HLine(y=1.).opts(color='red', line_dash='dashed', line_width=1)
hline_2 = hv.HLine(y=0.5).opts(color='red', line_dash='dashed', line_width=1)

In [ ]:
sig2 = sigmoid_curve * hline_0 * hline_1 * hline_2
sig2

In [ ]:
vline_0 = hv.VLine(x=0).opts(color='red', line_dash='dashed', line_width=1)

In [ ]:
sig2 * vline_0

In [ ]:
(sigmoid_curve * hline_0 * hline_1 * vline_0).opts(
    opts.Curve(width=600, padding=0.2)
)

### Exercise
- Load the [ice cream consumption data](https://tinyurl.com/vrtqbce) stored in `../data/icecream.csv` as a `pandas.DataFrame` object and plot different relaions as `hv.Curve` or `hv.Scatter`

- Suggestions: 
    - `temperature` vs. `consumption`
    - `income` vs. `consumption`
    - `price` vs. `consumption`

- Play with other elements and annotation elements

In [ ]:
## Exercise
import pandas as pd
data = pd.read_csv('../data/icecream.csv', index_col=0)
print(data.head())
# print(data.describe())

In [ ]:
## Soln
curve = hv.Curve(data, kdims=['temp', 'cons'])
scatter = hv.Scatter(data, kdims=['temp', 'cons']).opts(size=5, color='black')
# scatter = hv.Scatter(data, kdims=['temp', 'cons']).opts(size='income'), color='black')
# scatter = hv.Scatter(data, kdims=['temp', 'cons']).opts(size=hv.dim.norm('income')*10, color='black')
curve * scatter

### `hv.Image`
> a HoloViews ``Image`` allows you to view 2D arrays using an arbitrary color map. It is associated with a [2D coordinate system in continuous space](Continuous_Coordinates.ipynb), which is appropriate for values sampled from some underlying continuous distribution (as in a photograph or other measurements from locations in real space).  Slicing, sampling, etc. on an ``Image`` all use this continuous space, whereas the corresponding operations on a ``Raster`` work on the raw array coordinates.

In [ ]:
bounds=(-2,-3,5,2)   # Coordinate system: (left, bottom, top, right)
xs,ys = np.meshgrid(np.linspace(-2,5,50), np.linspace(2,-3, 30))
(hv.Image(np.sin(xs)+ys, bounds=bounds) 
 + hv.Image(np.sin(xs)+ys, bounds=bounds)[0:3, -2.5:2])

In [ ]:
# Read image data
img_fn = '../data/samples/cocolike/image/0000.jpg'
import skimage.io as skiio


In [ ]:
img = skiio.imread(img_fn) # read as uint8 RGB
img = img /255.
img.shape
plt.imshow(img)

In [ ]:
hv.Image(img[:,:,0], label='R')

### `hv.RGB`
It is a `hv.Image` equivalent for 3-channel RGB data

In [ ]:
x,y = np.meshgrid(np.linspace(-5,5,101), np.linspace(5,-5,101))
r = 0.5*np.sin(np.pi  +3*x**2+y**2)+0.5
g = 0.5*np.sin(x**2+2*y**2)+0.5
b = 0.5*np.sin(np.pi/2+x**2+y**2)+0.5

hv.RGB(np.dstack([r,g,b]))


In [ ]:
x = np.arange(img.shape[1])
y = np.arange(img.shape[0])[::-1]
bounds = (0, 0, img.shape[1], img.shape[0])
# hv.RGB(img, bounds=bounds, label='RGB')
hv_img = hv.RGB((x,y,img[:,:,0], img[:,:,1], img[:,:,2]),label='RGB')
hv_img 

### Exercise
- Read a corresponding mask file in `'../data/samples/cocolike/mask/0000.png'` 
and overlay the two images

- Try different alpha values

In [ ]:
### soln
mask = skiio.imread('../data/samples/cocolike/mask/0000.png')
print(mask.shape)
hv_mask = hv.Image(mask[:,:,0])
hv_mask


In [ ]:
 hv_img.opts(opts.RGB(shared_axes=False)) + hv_mask.opts(opts.Image(shared_axes=False))

## 2. Containers


### `hv.Overlay`
Use `*` to overlay multiple elements

### `hv.Layout`
Use `+` to overlay multiple elements

### `hv.HoloMap`
Parameter Spaces
>HoloViews also supplies container classes useful for visualizing parameter spaces or phase spaces, i.e., large collections of results for various combinations of parameters. These containers allow HoloViews to work with arbitrarily high-dimensional data, while having the underlying data held by Elements ensures that all of the data will be visualizable at every level of each data structure.



In [ ]:
frequencies =  np.linspace(0.5,2.0,5)
phases = np.linspace(0, np.pi*2, 5)
x,y = np.mgrid[-50:51, -50:51] * 0.1


def sine_array(phase, freq):
    return np.sin(phase + (freq*x**2+freq*y**2))

matrices = {(p, f): hv.Image(sine_array(p, f), label='Sinusoid Ring', group='Amplitude')
          for f in [0.5, 1.0,  1.5,  2.0]    # Frequencies
          for p in [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]}  # Phases

In [ ]:
hv.HoloMap(matrices, kdims=['phase', 'frequency'])

### `hv.DynamicMap`
Move on to next notebook: [01b-hv-dmap-gaussians.ipynb](./01b-hv-dmap-gaussians.ipynb)

# [SKIP] GeoViews Basics
GeoViews: HoloViews + Geographical operations

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

## 1. Projections

## 2.Elements
Geometric elements (similar to ESRI, QGIS)

### `gv.Points`

### `gv.Path`

### `gv.Tiles`

# Resources

1. HoloViews: 
- [Gallery](http://holoviews.org/reference/)
- [Documentation](http://holoviews.org/Reference_Manual/index.html)
- [User Guide](http://holoviews.org/user_guide/index.html)*
- [Tutorials](https://github.com/holoviz/holoviews/tree/master/doc/Tutorials)* 
- [Talks](#)


2. GeoViews
- [Website](http://geoviews.org/)
- [Gallery](http://geoviews.org/gallery/index.html), [User Guide](http://geoviews.org/user_guide/index.html)
---
`*`: highly recommended

